이 글은 pytorch를 이용해서 추천 시스템을 만드는 방법을 소개하고 있습니다. 
코드는 Jpub 출판 '파이토치 첫걸음'을 참고했습니다.

참고로 아래 코드는 구글 colab에서 작성되었습니다.코드 중간에 나오는 path 등은 코랩 기준입니다.

# 데이터 다운로드

추천 시스템에 사용되는 대표적인 데이터 셋인 무비렌즈 데이터를 이용할 것입니다.

데이터에 대한 설명은 https://grouplens.org/datasets/movielens/ 에서 보실 수 있습니다.

데이터는 리눅스라면 아래처럼 wget을 이용해 가져올 수도 있습니다.

아니면 http://files.grouplens.org/datasets/movielens/ml-20m.zip 에 들어가면 압축파일이 다운받아집니다.

In [1]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip

--2019-09-10 03:46:57--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  47.8MB/s    in 4.4s    

2019-09-10 03:47:01 (43.1 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


# 데이터 임포트

압축 파일을 열면 ratings.csv란 파일이 있습니다. 이 파일 안에 영화에 대한 유저별 평가 데이터가 들어있습니다.

In [2]:
import pandas as pd

df = pd.read_csv("/content/ml-20m/ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


데이터는 유저ID, 영화ID, 평점, 평가한 시간입니다.
평가 시간은 1970년 1월 1일 기준으로 경과한 초 단위 시간을 나타냅니다. 
이 튜토리얼에서는 시간 데이터는 이용하지 않을 것입니다.

In [3]:
# 200만 건이 넘는 데이터입니다.
df.shape

(20000263, 4)

In [4]:
# 평가 점수는 0.5점 단위로 0.5에서 5점까지로 구성되어 있습니다.
sorted(df['rating'].unique())

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

지금 만들려고 하는 것은 추천 시스템입니다. 말하자면 A유저가 좋아할 만한 영화를 추천하고자 하는 것입니다. 그리고 '좋아한다'의 기준이 평점이라 한다면 문제를 다음과 같이 재정의할 수 있을 것입니다.

**유저ID와 영화ID를 입력으로 받아서 해당 영화에 대한 평점을 예측하는 문제**

그러므로 평점이 y(레이블)에 해당하고, 유저ID와 영화ID가 x1, x2가 됩니다.

사이킷런의 train_test_split 함수를 통해 레이블과 테스트 데이터를 분리하겠습니다.


In [0]:
from sklearn.model_selection import train_test_split
data = df[['userId', 'movieId']].values
target = df['rating'].values

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1, shuffle=True)

In [6]:
# 훈련 데이터에 약 180만 개의 데이터가 할달되었습니다.
X_train.shape

(18000236, 2)

이제 데이터를 파이토치 모델에 넣을 수 있는 형태로 바꾸는 과정입니다.

파이토치 텐서 객체로 변환해주고, X값과 y값을 묶어서 데이터셋 객체로 만들어줍니다.

저는 무비렌즈 데이터가 크기 때문에, 훈련 데이터를 반으로 쪼개서 훈련 세트를 두 개 만들어주었습니다.

In [0]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn, optim

train_dataset1 = TensorDataset(torch.tensor(X_train[:9000000], dtype=torch.int64), 
                               torch.tensor(y_train[:9000000], dtype=torch.float32))
train_dataset2 = TensorDataset(torch.tensor(X_train[9000000:], dtype=torch.int64), 
                               torch.tensor(y_train[9000000:], dtype=torch.float32))

train_loader1 = DataLoader(train_dataset1, batch_size=1024, shuffle=True)
train_loader2 = DataLoader(train_dataset2, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.int64), 
                             torch.tensor(y_test, dtype=torch.float32))
test_loader = DataLoader(test_dataset, batch_size=1024)

# 모델 만들기

여기서는 모델을 두 개 만들어볼 것입니다.

하나는 유저ID와 무비ID를 받아서 영화에 대한 평점을 예측하는 모델이고,다른 하나는 영화 장르 정보까지 이용해서 평점을 예측하는 모델입니다. 이용하는 정보만 추가되었을 뿐 같은 모델입니다.

방법은 다음과 같습니다.

1. 유저ID별로 길이가 user_k인 벡터를 하나씩 부여합니다.

2. 영화ID도 마찬가지로 길이가 item_k인 벡터를 부여합니다.

3. 유저ID 벡터와 영화ID 벡터를 합쳐서 길이가 user_k + item_k인 벡터를 만듭니다.

4. 그렇게 만들어진 벡터를 은닉층이 하나인 완전연결 층에 넣습니다.

5. 마지막 층의 활성화함수는 시그모이드를 사용했습니다. 그러나 영화 평점은 5점까지므로 곱하기 5를 해줍니다.

여기서 1번 단계 구현을 위해 사용하는 레이어가 임베딩 레이어입니다. 임베딩 레이어는 인덱스(정수)를 받아서 원하는 길이만큼의 벡터를 반환해줍니다.

모델에 인풋으로 넣어주는 것은 크기가 (batch size, 2)인 텐서입니다. 텐서의 첫 번째 열은 유저ID고 두 번째 열은 영화ID입니다.

인풋으로 들어간 텐서는 크기가 (batch size,)인 텐서로 쪼개집니다. 두 개의 텐서는 각각 유저ID와 영화ID를 표현합니다.

쪼개진 텐서를 이제 각각 벡터로 변환해줘야 합니다. 각각 유저ID를 벡터로 변환해주는 유저ID 임베딩 레이어와 영화ID를 벡터로 변환해주는 영화ID 임베딩 레이어로 집어넣어줍니다.임베딩 레이어를 통과하면 각각 (batch size, user_k), (batch size, item_k) 크기의 텐서가 만들어지고, 이를 하나로 합쳐서 (batch size, user_k + item_k) 크기의 텐서로 만들어줍니다.

이제 인풋 크기가 user_k+item_k이고 아웃풋이 1인 완전연결 층에 넣어주어 모델을 학습시키면 됩니다.그러면 완전연결 층의 파라미터뿐만 아니라 임베딩 층의 파라미터까지 학습이 이루어져서, 각각 유저와 각각의 영화에 해당하는 벡터 표현을 구할 수 있습니다. 

In [0]:
# 사용할 모델

class NeuralMatrixFactorization(nn.Module):
  def __init__(self, max_user, max_item, user_k=10, item_k=10, hidden_dim=50):
    super().__init__()
    self.user_emb = nn.Embedding(max_user, user_k, 0)
    self.item_emb = nn.Embedding(max_item, item_k, 0)
    self.mlp = nn.Sequential(
        nn.Linear(user_k + item_k, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, 1)
    )
  def forward(self, x):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx) # (batch_size, user_k)
    item_feature = self.item_emb(item_idx) # (batch_size, item_k)
    
    out = torch.cat([user_feature, item_feature], 1) # (batch_size, user_k+item_k)
    out = self.mlp(out)  # (batch_size, 1)
    out = torch.sigmoid(out) * 5
    return out.squeeze()   # (batch_size,)

In [0]:
# 평가함수 작성

def eval_net(net, loader, score_fn=nn.functional.l1_loss, device='cpu'): # 훈련은 MSE로 하되, 평가는 해석하기가 쉬운 l1_loss를 사용했습니다.
  ys = []
  y_preds = []
  for x, y in loader:
    x = x.to(device)
    ys.append(y)
    with torch.no_grad():
      y_pred = net(x).cpu()
    y_preds.append(y_pred)
  score = score_fn(torch.cat(ys).squeeze(), torch.cat(y_preds))
  return score.item()

In [0]:
# 훈련 부분 작성하기
from statistics import mean

def train_net(net, loss_fn, optimizer, loader, device='cpu'):
  losses = []
  for x, y in loader:
    x = x.to(device)
    y = y.to(device)
    out = net(x)
    loss = loss_fn(out, y)
    net.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
  return mean(losses)

In [43]:
# 훈련하기

device = "cuda" if torch.cuda.is_available() else "cpu"

max_user, max_item = data.max(0) # 유저ID와 영화ID는 1부터 시작해서 1씩 증가합니다. 그러므로 max를 취해주면 유저ID와 영화ID 개수가 반환됩니다.
max_user = int(max_user) + 1     # 새로운 유저나 영화가 인풋으로 들어올 것에 대비해 +1을 해줍니다.
max_item = int(max_item) + 1

net = NeuralMatrixFactorization(max_user, max_item)
net.cuda()
optimizer = optim.Adam(net.parameters(), lr=0.01) # 데이터가 크기 때문에 넓은 공간을 탐색하기 쉽도록 학습률을 크게 합니다.
loss_fn = nn.MSELoss()


for epoch in range(3): # 시간이 오래 걸려서 에폭 3으로 했습니다.
  
  net.train()
  loss = train_net(net, loss_fn, optimizer, train_loader1, device=device)
  net.eval()
  test_score = eval_net(net, test_loader, device=device)
  print(epoch, loss, test_score)
  
  net.train()
  loss = train_net(net, loss_fn, optimizer, train_loader2, device=device)
  net.eval()
  test_score = eval_net(net, test_loader, device=device)
  print(epoch, loss, test_score)

0 0.785351644861549 0.6597837805747986
0 0.7191154012772275 0.640923023223877
1 0.6957194821690807 0.6368798613548279
1 0.6823071704111544 0.6314395666122437
2 0.673727889462407 0.6285171508789062
2 0.6675265018375796 0.6270928382873535


# 장르 정보 이용

영화의 장르 정보는 movies.csv 파일에 별도로 저장되어 있습니다.

In [12]:
genre_data = pd.read_csv("/content/ml-20m/movies.csv")
genre_data.head()  

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


타이틀에 연도 정보까지 있네요. 저희는 장르 정보만 이용할 것입니다.

하나의 영화가 여러 장르에 속할 경우 '|' 표시로 구분되고 있습니다. 

장르 정보를 이용하는 방식은 다음과 같습니다.

이전에 저는 영화ID마다 하나의 벡터를 배정해주었습니다.

이번에도 똑같이 영화ID마다 하나의 벡터를 배정해주되, 배정되는 벡터에서 일부 차원은 영화의 장르 정보를 담고 있도록 만드는 겁니다.

이를 위해서 장르 정보만을 위한 임베딩 레이어를 따로 만들어 학습할 수도 있지만, 여기서는 속하는 장르의 차원은 값이 1이고 속하지 않는 장르 차원은 값이 0인 벡터로 장르 정보를 표현해주겠습니다.


이를 위해 사이킷런의 CountVectorizer 모듈을 이용하겠습니다. CountVectorizer는 전체 텍스트를 조사해 단어 사전을 구축한 뒤, 각각의 문서를 단어 출현 빈도 벡터로 변환해줍니다.

예를 들어, 어떤 영화가 로멘스, 액션, 판타지에 속하고 코메디는 아니라면, [1, 1, 1, 0]으로 표현할 수 있을 것입니다. 이를 영화 임베딩 레이어를 통과한 벡터와 합치면 결국 장르 정보까지도 표현하는, 영화ID를 표현하는 벡터 더 큰 차원의 벡터 표현이 만들어지게 됩니다.

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

movie_id = genre_data['movieId']
genres = genre_data['genres']

count_vec = CountVectorizer()
count_vec.fit(genres)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

24개의 장르가 있는 것을 볼 수 있습니다.

In [14]:
count_vec.vocabulary_

{'action': 0,
 'adventure': 1,
 'animation': 2,
 'children': 3,
 'comedy': 4,
 'crime': 5,
 'documentary': 6,
 'drama': 7,
 'fantasy': 8,
 'fi': 9,
 'film': 10,
 'genres': 11,
 'horror': 12,
 'imax': 13,
 'listed': 14,
 'musical': 15,
 'mystery': 16,
 'no': 17,
 'noir': 18,
 'romance': 19,
 'sci': 20,
 'thriller': 21,
 'war': 22,
 'western': 23}

장르 정보를 텐서로 바꾸어줍니다. 그리고 영화ID를 넣으면 해당 영화의 장르 벡터가 반환되도록 사전을 만들어줍니다.

In [0]:
num_genres = len(count_vec.vocabulary_)
genre_vec = count_vec.transform(genres).toarray()
genre_vec = torch.tensor(genre_vec, dtype=torch.float32)
genre_dict = dict(zip(movie_id, genre_vec))

## 커스텀 데이터 로더 만들기

모델은 두 가지 방식으로 구현이 가능합니다.

하나는 유저ID와 영화ID를 넣으면 모델 내에서 각각의 ID에 해당하는 벡터 표현을 가져와 완전연결 층의 입력으로 전달해주는 것입니다.

다른 하나는 모델 입력으로 (유저ID, 영화ID, 장르 정보)의 쌍을 넣는 것입니다. 

아래 코드는 후자를 선택했습니다. 모델에 입력으로 넣을 데이터에 장르 정보(해당 영화ID에 해당하는 장르 텐서)가 추가되었으니, 데이터 로더를 새롭게 구성해줍니다.

In [0]:
from torch.utils.data import Dataset

class MovieLensDataset(Dataset):
  def __init__(self, x, y, genre_dict):
    assert len(x) == len(y)
    self.x = x
    self.y = y
    self.genre_dict = genre_dict
    
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    x = self.x[idx]
    y = self.y[idx]
    g = self.genre_dict.get(x[1].item()) # 새로운 데이터에 대해 실험할 때는 보지 못한 유저ID나 영화ID가 나올 것에 대비해야 합니다. 
                                         # 여기서는 이 부분을 처리하지 않았습니다.
    return x, y, g # 유저ID, 영화ID, 장르 텐서

In [0]:
train_dataset1 = MovieLensDataset(
    torch.tensor(X_train[:9000000], dtype=torch.int64),
    torch.tensor(y_train[:9000000], dtype=torch.float32),
    genre_dict
)
train_dataset2 = MovieLensDataset(
    torch.tensor(X_train[9000000:], dtype=torch.int64),
    torch.tensor(y_train[9000000:], dtype=torch.float32),
    genre_dict
)
test_dataset = MovieLensDataset(
    torch.tensor(X_test, dtype=torch.int64),
    torch.tensor(y_test, dtype=torch.float32),
    genre_dict
)

train_loader1 = DataLoader(train_dataset1, batch_size=1024, shuffle=True)
train_loader2 = DataLoader(train_dataset2, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024)

## 모델 작성

모델은 완전연결 층의 인풋 크기가 장르 개수만큼 늘어난 것을 제외하고는 똑같습니다.

In [0]:
class NeuralMatrixFactorization2(nn.Module):
  def __init__(self, max_user, max_item, num_genres, user_k=10, item_k=10, hidden_dim=50):
    super().__init__()
    self.user_emb = nn.Embedding(max_user, user_k, 0)
    self.item_emb = nn.Embedding(max_item, item_k, 0)
    self.mlp = nn.Sequential(
        nn.Linear(user_k+item_k+num_genres, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, 1)
    )
  def forward(self, x, g):
    user_feature = self.user_emb(x[:,0]) # (batch_size, user_k)
    item_feature = self.item_emb(x[:,1]) # (batch_size, item_k)
    out = torch.cat([user_feature, item_feature, g], 1)
    out = self.mlp(out)
    out = torch.sigmoid(out) * 5
    return out.squeeze()

In [0]:
# 평가함수 작성

def eval_net(net, loader, score_fn=nn.functional.l1_loss, device='cpu'):
  ys = []
  y_preds = []
  for x, y, g in loader:
    x = x.to(device)
    g = g.to(device)
    ys.append(y)
    with torch.no_grad():
      y_pred = net(x, g).cpu()
    y_preds.append(y_pred)
  score = score_fn(torch.cat(ys).squeeze(), torch.cat(y_preds))
  return score.item()

In [0]:
# 훈련 부분 작성하기
from statistics import mean

def train_net(net, loss_fn, optimizer, loader, device='cpu'):
  losses = []
  for x, y, g in loader:
    x = x.to(device)
    y = y.to(device)
    g = g.to(device)
    out = net(x, g)
    loss = loss_fn(out, y)
    net.zero_grad()
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
  return mean(losses)

In [26]:
# 훈련하기

device = "cuda" if torch.cuda.is_available() else "cpu"

max_user, max_item = data.max(0)
max_user = int(max_user) + 1
max_item = int(max_item) + 1

net = NeuralMatrixFactorization2(max_user, max_item, num_genres)
net.cuda()
optimizer = optim.Adam(net.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

for epoch in range(3):
  
  net.train()
  loss = train_net(net, loss_fn, optimizer, train_loader1, device=device)
  net.eval()
  test_score = eval_net(net, test_loader, device=device)
  print(epoch, loss, test_score)
  
  net.train()
  loss = train_net(net, loss_fn, optimizer, train_loader2, device=device)
  net.eval()
  test_score = eval_net(net, test_loader, device=device)
  print(epoch, loss, test_score)

0 0.7842741769735316 0.6532257795333862
0 0.716683269774412 0.6464508771896362
1 0.6908000394617195 0.633633553981781
1 0.672769879745542 0.6257384419441223
2 0.6563715703590467 0.6199726462364197
2 0.6422819652248162 0.6206002831459045


장르 정보까지 이용하자 손실이 조금 더 낮아진 걸 확인할 수 있습니다.